In [1]:
import numpy as np
import pandas as pd
import altair as alt

from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
from clickhouse_driver import Client as Clickhouse

def click_query(q, **kw):
    click = Clickhouse("localhost")
    return click.query_dataframe(q, params=kw)

In [3]:
click_query("DESCRIBE TABLE obs_web")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,probe_asn,Int32,,,,,
9,probe_cc,String,,,,,


In [4]:
click_query("DESCRIBE TABLE obs_web_ctrl")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,hostname,String,,,,,
9,observation_id,String,,,,,


In [35]:
MEASUREMENT_START_DAY = '2024-04-01'
MEASUREMENT_END_DAY = '2024-07-01'
ANALYSIS_COUNTRY_CODES = [
    "ID"
]

In [6]:
%%time
df = click_query("""
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
dns_answer_type,
http_request_url,
http_failure,
http_runtime,
http_response_body_sha1,
http_response_status_code,
http_response_body_length,
http_response_body_is_truncated,
probe_analysis
FROM obs_web
WHERE probe_cc = 'ID' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 20.4 s, sys: 1.68 s, total: 22.1 s
Wall time: 1min 31s


In [7]:
pd.set_option('display.max_colwidth', None)
df['explorer_url'] = "https://explorer.ooni.org/measurement/" + df['measurement_uid']

In [8]:
df.to_csv('2024-ID-df.csv')

In [9]:
df = pd.read_csv('2024-ID-df.csv')

In [10]:
df[['probe_asn', 'probe_as_org_name']].drop_duplicates(subset=['probe_asn']).to_csv('2024_ID_ASN_Lookup.csv')

In [11]:
len(df)

728363

In [12]:
df

,Unnamed: 0,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
0,0,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://twitter.com/,20240401002241.289265_ID_webconnectivity_227fd54c442ba836,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:40,twitter.com,web_connectivity,104.244.42.193,443.0,13414.0,Twitter Inc.,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,system,NaN,104.244.42.193,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401002241.289265_ID_webconnectivity_227fd54c442ba836
1,1,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://twitter.com/,20240401002241.289265_ID_webconnectivity_227fd54c442ba836,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:40,twitter.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://twitter.com/,NaN,NaN,8b7568f04c7e0d071570b1fbff57b348c0188a0b,200.0,131072.0,1.0,false,https://explorer.ooni.org/measurement/20240401002241.289265_ID_webconnectivity_227fd54c442ba836
2,2,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.facebook.com/,20240401002242.717887_ID_webconnectivity_563dad58be9950f5,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:42,www.facebook.com,web_connectivity,157.240.15.35,443.0,32934.0,"Facebook, Inc.",103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,system,NaN,157.240.15.35,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401002242.717887_ID_webconnectivity_563dad58be9950f5
3,3,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.facebook.com/,20240401002242.717887_ID_webconnectivity_563dad58be9950f5,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:42,www.facebook.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://www.facebook.com/,NaN,NaN,dd7d78e25435346d5d965d023e2a7a2911e4b57a,200.0,57422.0,0.0,false,https://explorer.ooni.org/measurement/20240401002242.717887_ID_webconnectivity_563dad58be9950f5
4,4,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.instagram.com/,20240401002245.667553_ID_webconnectivity_cdd2895b8ed9f531,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:45,www.instagram.com,web_connectivity,57.144.100.34,443.0,32934.0,"Facebook, Inc.",103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,system,NaN,57.144.100.34,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401002245.667553_ID_webconnectivity_cdd2895b8ed9f531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728358,728358,20240425T125024Z_webconnectivity_ID_4761_n1_skLTe4yAnxHVKwNF,https://www.100webspace.com/,20240425125951.235621_ID_webconnectivity_ce298f454276b168,ID,4761,PT. INDOSAT Tbk,ID,unknown,2024-04-25 12:59:47,www.100webspace.com,web_connectivity,198.23.53.13,443.0,32748.0,Steadfast,180.241.233.195,ID,7713,Telekomunikasi Indonesia (PT),ID,system,NaN,198.23.53.13,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240425125951.235621_ID_webconnectivity_ce298f454276b168
728359,728359,20240425T125024Z_webconnectivity_ID_4761_n1_skLTe4yAnxHVKwNF,https://www.100webspace.com/,20240425125951.235621_ID_webconnectivity_ce298f454276b168,ID,4761,PT. INDOSAT Tbk,ID,unknown,2024-04-25 12:59:47,www.100webspace.com,web_connectivity,NaN,NaN,NaN,NaN,180.2

In [29]:
df['probe_analysis'].value_counts()

false           663699
dns              20589
http-failure     14801
tcp_ip            6808
http-diff         3558
Name: probe_analysis, dtype: int64

In [14]:
df_dns_answer = df[df['dns_answer_type']=='A'].drop_duplicates(subset=['measurement_uid'])

In [15]:
len(df_dns_answer)

181654

In [16]:
dns_resp_sorted = df_dns_answer[
    (df_dns_answer['probe_analysis'] == 'dns')
].groupby('dns_answer')['hostname'].nunique().sort_values().reset_index()

In [17]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 2
]

,dns_answer,hostname
100,3.130.253.23,3
101,3.130.204.160,3
102,3.94.41.167,3
103,185.107.56.193,3
104,54.209.32.212,3
105,18.119.154.66,3
106,115.178.182.58,4
107,116.206.10.31,7
108,10.93.93.20,10
109,202.62.8.233,11


In [18]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 10
]

,dns_answer,hostname
109,202.62.8.233,11
110,202.62.8.232,12
111,36.86.63.182,18
112,103.189.63.245,18
113,182.23.79.195,35
114,118.98.96.151,37
115,103.154.123.132,39
116,139.255.196.196,41
117,203.119.13.76,45
118,114.6.128.8,53


In [19]:
dns_suspicious = dns_resp_sorted[dns_resp_sorted['hostname'] > 10]['dns_answer'].values.tolist()
dns_suspicious


['202.62.8.233',
 '202.62.8.232',
 '36.86.63.182',
 '103.189.63.245',
 '182.23.79.195',
 '118.98.96.151',
 '103.154.123.132',
 '139.255.196.196',
 '203.119.13.76',
 '114.6.128.8',
 '203.119.13.75',
 '202.169.44.80',
 '0.0.0.0',
 '75.2.41.216',
 '36.86.63.185',
 '202.3.218.137',
 '208.91.112.55',
 '202.137.1.74']

In [20]:
df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)].drop_duplicates(subset=['dns_answer']).to_csv('2024_ID_dns_suspicious.csv')

In [21]:
dns_answer_filter = df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)]

dns_answer_filter = dns_answer_filter.pivot_table(index=['dns_answer', 'probe_cc'], 
              values=['hostname'],
              aggfunc={'hostname': ['nunique', lambda x: ', '.join(set(x))]}).rename(columns = {
                                                                    '<lambda_0>': 'hostnames',
                                                                    'len' : 'hostname_count'}).reset_index()

dns_answer_filter.head()

dns_answer probe_cc  \
                              
0          0.0.0.0       ID   
1  103.154.123.132       ID   
2   103.189.63.245       ID   
3      114.6.128.8       ID   
4    118.98.96.151       ID   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [22]:
dns_answer_filter.to_csv('2024_ID_dns_answer_hostname')

In [23]:
df_http_answer = df[df['http_request_url'].notnull()].drop_duplicates(subset=['measurement_uid'])

In [24]:
df_http_answer.head()

,Unnamed: 0,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
1,1,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://twitter.com/,20240401002241.289265_ID_webconnectivity_227fd54c442ba836,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:40,twitter.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://twitter.com/,NaN,NaN,8b7568f04c7e0d071570b1fbff57b348c0188a0b,200.0,131072.0,1.0,false,https://explorer.ooni.org/measurement/20240401002241.289265_ID_webconnectivity_227fd54c442ba836
3,3,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.facebook.com/,20240401002242.717887_ID_webconnectivity_563dad58be9950f5,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:42,www.facebook.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://www.facebook.com/,NaN,NaN,dd7d78e25435346d5d965d023e2a7a2911e4b57a,200.0,57422.0,0.0,false,https://explorer.ooni.org/measurement/20240401002242.717887_ID_webconnectivity_563dad58be9950f5
5,5,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.instagram.com/,20240401002245.667553_ID_webconnectivity_cdd2895b8ed9f531,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:45,www.instagram.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://www.instagram.com/,NaN,NaN,6fd1aefe1767655e47f7128c0f656da1fa06b24f,200.0,131072.0,1.0,false,https://explorer.ooni.org/measurement/20240401002245.667553_ID_webconnectivity_cdd2895b8ed9f531
14,14,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.youtube.com/,20240401002247.512238_ID_webconnectivity_edbae59eeda78a77,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:47,www.youtube.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://www.youtube.com/,NaN,NaN,030653416373bed2d62096c563564b327fd6388e,200.0,131074.0,1.0,false,https://explorer.ooni.org/measurement/20240401002247.512238_ID_webconnectivity_edbae59eeda78a77
20,20,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,http://www.no-ip.com/,20240401002256.581560_ID_webconnectivity_6b1379602d5bacc4,ID,150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:56,www.noip.com,web_connectivity,NaN,NaN,NaN,NaN,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,NaN,NaN,NaN,NaN,https://www.noip.com/,NaN,NaN,fb280a6efefc1d8afb62f3c8c74b16c83b6f12ab,200.0,80965.0,0.0,false,https://explorer.ooni.org/measurement/20240401002256.581560_ID_webconnectivity_6b1379602d5bacc4


In [30]:
http_domain_count = df_http_answer[
    df_http_answer['probe_analysis'] == 'http-diff'
].groupby('http_response_body_sha1')['hostname'].nunique().sort_values().reset_index()

In [31]:
http_domain_count[
    http_domain_count['hostname'] > 2
]

,http_response_body_sha1,hostname
300,0eece0edce7f2bec32e99c26a0ba7789bcb3b4b1,4


In [32]:
http_domain_count[
    http_domain_count['hostname'] > 10
]

,http_response_body_sha1,hostname


In [33]:
http_suspicious = http_domain_count[http_domain_count['hostname'] > 10]['http_response_body_sha1'].values.tolist()
http_suspicious


[]

In [34]:
df_http_answer[df_http_answer['http_response_body_sha1'].isin(http_suspicious)].drop_duplicates(subset=['http_response_body_sha1']).to_csv('2024_ID_http_suspicious.csv')

In [36]:
confirmed_ips = [
    '36.86.63.185',
    '202.137.1.74',
    '27.54.116.70',
    '202.169.44.80',
    '114.6.128.8','75.2.41.216',
    '202.152.4.67',
    '103.169.16.2',
    '110.50.83.31',
    '202.3.218.137',
    '203.119.13.76',
    '116.206.10.31',
    '115.178.182.58',
    '158.140.186.3',
    '203.119.13.75',
    '99.83.188.13',
    '49.128.177.13',
    '150.107.140.200',
    '202.89.117.64',
    '202.56.160.131',
    '202.56.160.132',
    '203.99.130.131',
    '103.124.138.6',
    '210.210.178.70',
    '114.129.23.9',
    '103.88.170.170',
    '27.123.220.197',
    '103.94.168.168',
    '119.18.152.218',
    '113.197.108.236',
    '103.14.16.18',
    '103.170.115.114'
]

likely_ips = [
    '202.62.8.233','202.62.8.232','36.86.63.182','118.98.96.151','103.154.123.132','139.255.196.196','202.70.53.123','182.23.79.195','121.101.130.122','103.47.134.195','103.47.132.195','202.165.36.42','101.255.0.226','101.255.1.226','202.58.203.201','103.123.248.32','118.98.97.151']

false_positive_ips = [
    '103.144.182.26','38.60.171.148','208.91.112.55','144.208.225.159','45.114.118.156','34.101.84.129','147.139.211.126','103.87.68.66','146.112.61.106'
]

confirmed_http_resp = [
    '0eece0edce7f2bec32e99c26a0ba7789bcb3b4b1','e213805c85233a4d38f729684912db3434e5bb64'
]

likely_http_resp = [
    'eb6ce822e8e9989755037c6838f3b100fd6815c3','c7fe29c30d44b120412496b4ca8a2a1a9233c515','5a24aed87424875c0436b60ff74399aed8bb0118','ba8b89809b447c65a32345f75c7a3344df13874d','3a60ac61b3489264e8bada3ac797316c0664ceb3','fd1ac680d6db2d93c1c14e745b46c6658fb96173','a13e4881f5c435952735152b69452d8cbe8c40a6','86783790fadceaf3f49b55e63ae5cd04d566de07','b2985810660b99d38fdca730fa744cdfa12a0277','ba33ee47806d8d458dbdfc390f4d73c6f9dc5e6c','0142b1020eb40f2f7bba98aa9be295027b92cc1f','7d5a91a063b730fe32395b45b3d5dfcb3e93132f','49b0f8670844bd149fdefc4abacb4a79d6c22e2e','f4e50edfc5c681c07571b6e4dd406c3764bfaf52','841ff3e0ab0885154c14000f157ed15bec9a21eb'
]

false_positive_http_resp = [
]


In [39]:
%%time
df_with_failure = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'ID' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 46.7 s, sys: 3.07 s, total: 49.8 s
Wall time: 3min 21s


In [40]:
pd.set_option('display.max_colwidth', None)
df_with_failure['explorer_url'] = "https://explorer.ooni.org/m/" + df_with_failure['measurement_uid']

In [41]:
df_with_failure["probe_asn"] = "AS" + df_with_failure["probe_asn"].apply(str)

In [42]:
df_with_failure

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_failure,http_response_body_sha1,http_request_url,http_failure,failure_class,failure_str_full,failure_str,failure_str_raw,explorer_url
0,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://twitter.com/,20240401002241.289265_ID_webconnectivity_227fd54c442ba836,ID,AS150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:40,twitter.com,web_connectivity,104.244.42.193,443.0,13414.0,Twitter Inc.,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,system,None,104.244.42.193,1.0,None,0.104272,2.0,4.0,3501.0,389.0,write_4,1,1.0,"CN=twitter.com,O=Twitter\, Inc.,L=San Francisco,ST=California,C=US",twitter.com,"CN=DigiCert Global G2 TLS RSA SHA256 2020 CA1,O=DigiCert Inc,C=US",DigiCert Global G2 TLS RSA SHA256 2020 CA1,"[twitter.com, www.twitter.com, x.com, www.x.com]",2024-10-31 23:59:59,2023-11-02 00:00:00,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401002241.289265_ID_webconnectivity_227fd54c442ba836
1,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://twitter.com/,20240401002241.289265_ID_webconnectivity_227fd54c442ba836,ID,AS150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:40,twitter.com,web_connectivity,None,NaN,NaN,None,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,8b7568f04c7e0d071570b1fbff57b348c0188a0b,https://twitter.com/,None,ok,ok,,,https://explorer.ooni.org/m/20240401002241.289265_ID_webconnectivity_227fd54c442ba836
2,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.facebook.com/,20240401002242.717887_ID_webconnectivity_563dad58be9950f5,ID,AS150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:42,www.facebook.com,web_connectivity,157.240.15.35,443.0,32934.0,"Facebook, Inc.",103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,system,None,157.240.15.35,1.0,None,0.023506,3.0,4.0,3245.0,394.0,write_4,1,1.0,"CN=*.facebook.com,O=Meta Platforms\, Inc.,L=Menlo Park,ST=California,C=US",*.facebook.com,"CN=DigiCert SHA2 High Assurance Server CA,OU=www.digicert.com,O=DigiCert Inc,C=US",DigiCert SHA2 High Assurance Server CA,"[*.facebook.com, *.facebook.net, *.fbcdn.net, *.fbsbx.com, *.m.facebook.com, *.messenger.com, *.xx.fbcdn.net, *.xy.fbcdn.net, *.xz.fbcdn.net, facebook.com, messenger.com]",2024-04-08 23:59:59,2024-01-09 00:00:00,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401002242.717887_ID_webconnectivity_563dad58be9950f5
3,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.facebook.com/,20240401002242.717887_ID_webconnectivity_563dad58be9950f5,ID,AS150249,PT Atharva Telematika Persada,ID,unknown,2024-04-01 00:22:42,www.facebook.com,web_connectivity,None,NaN,NaN,None,103.87.68.194,ID,150249,PT Atharva Telematika Persada,ID,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,dd7d78e25435346d5d965d023e2a7a2911e4b57a,https://www.facebook.com/,None,ok,ok,,,https://explorer.ooni.org/m/20240401002242.717887_ID_webconnectivity_563dad58be9950f5
4,20240401T002238Z_webconnectivity_ID_150249_n1_ZwKCzr33VUNdYn7d,https://www.instagram.com/,20240401002245.667553_ID_webconnectivit

In [43]:
df_with_failure.to_csv('2024-ID-df_with_failure.csv')

In [ ]:
#df_with_failure = pd.read_csv('2024-ID-df_with_failure.csv')

In [44]:
len(df_with_failure)

728363

In [45]:
list(df_with_failure)

['report_id',
 'input',
 'measurement_uid',
 'probe_cc',
 'probe_asn',
 'probe_as_org_name',
 'probe_as_cc',
 'network_type',
 'measurement_start_time',
 'hostname',
 'test_name',
 'ip',
 'port',
 'ip_asn',
 'ip_as_org_name',
 'resolver_ip',
 'resolver_cc',
 'resolver_asn',
 'resolver_as_org_name',
 'resolver_as_cc',
 'dns_engine',
 'dns_failure',
 'dns_answer',
 'tcp_success',
 'tcp_failure',
 'tls_handshake_time',
 'tls_handshake_read_count',
 'tls_handshake_write_count',
 'tls_handshake_read_bytes',
 'tls_handshake_write_bytes',
 'tls_handshake_last_operation',
 'tls_success',
 'tls_is_certificate_valid',
 'tls_end_entity_certificate_subject',
 'tls_end_entity_certificate_subject_common_name',
 'tls_end_entity_certificate_issuer',
 'tls_end_entity_certificate_issuer_common_name',
 'tls_end_entity_certificate_san_list',
 'tls_end_entity_certificate_not_valid_after',
 'tls_end_entity_certificate_not_valid_before',
 'tls_failure',
 'http_response_body_sha1',
 'http_request_url',
 'http

In [71]:
# see: https://learn.microsoft.com/en-us/windows/win32/winsock/windows-sockets-error-codes-2
unknown_failure_map = {
    ': server misbehaving': 'dns_server_misbehaving',
    ': read: connection refused': 'connection_refused',
    ': connect: network is unreachable': 'network_unreachable',
    'tls: first record does not look like a TLS handshake': 'tls_bad_first_record',
    'remote error: tls: handshake failure': 'tls_handshake_failure',
    'remote error: tls: illegal parameter': 'tls_illegal_parameter',
    'connectex: No connection could be made because the target machine actively refused it': 'connection_refused',
    'read: connection refused': 'connection_refused',
    'remote error: tls: access denied': 'tls_access_denied',
    'remote error: tls: internal error': 'tls_internal_error',
    'HTTP/1.x transport connection broken: malformed HTTP version': 'http_malformed_response',
    'net/http: timeout awaiting response headers': 'http_timeout',
    'read: operation timed out': 'timed_out',
    'connect: operation timed out': 'timed_out',
    ': No address associated with hostname': 'dns_nxdomain_error',
    ': connect: bad file descriptor': 'bad_file_descriptor',
    'stream error: stream ID': 'http_stream_error',
    
    # This looks more like a golang-bug: https://github.com/golang/go/issues/31259
    'readLoopPeekFailLocked: <nil>': 'http_golang_bug',

    ': connect: no route to host': 'host_unreachable',
    ': connect: cannot assign requested address': 'address_not_available',
    'getaddrinfow: The requested name is valid, but no data of the requested type was found.': 'dns_no_answer',
    'wsarecv: Se ha forzado la interrupción de una conexión existente por el host remoto.': 'connection_reset',
    'wsarecv: An existing connection was forcibly closed by the remote host.': 'connection_reset',
    'wsarecv: Connessione in corso interrotta forzatamente dall\'host remoto.': 'connection_reset',
    'wsarecv: Uma ligação existente foi forçada a fechar pelo anfitrião remoto': 'connection_reset',
    
    'getaddrinfow: Ceci est habituellement une erreur temporaire qui se produit durant la résolution du nom d’hôte et qui signifie que le serveur local n’a pas reçu de réponse d’un serveur faisant autorité': 'dns_temporary_failure',
    'getaddrinfow: Dies ist normalerweise ein zeitweiliger Fehler bei der Auflösung von Hostnamen. Grund ist, dass der lokale Server keine Rückmeldung vom autorisierenden Server erhalten hat.': 'dns_temporary_failure',
    'getaddrinfow: Este é geralmente um erro temporário durante a resolução de nomes de anfitrião e significa que o servidor local não recebeu uma resposta de um servidor autoritário': 'dns_temporary_failure',
    'getaddrinfow: Éste es normalmente un error temporal durante la resolución de nombres de host y significa que el servidor local no recibió una respuesta de un servidor autoritativo': 'dns_temporary_failure',
}
def map_unknown_failure(failure_str):
    if not failure_str.startswith("unknown_failure"):
        return failure_str
    for substring, clean_failure in unknown_failure_map.items():
        if substring in failure_str:
            return clean_failure
    return "unknown_failure"

def simplify_failure(failure_str):
    if failure_str in ['timed_out', 'generic_timeout_error', 'deferred_timeout_error']:
        return 'timeout'
    
    if failure_str in ['android_dns_cache_no_data', 'dns_nxdomain_error']:
        return 'nxdomain'
    
    if failure_str in ['connection_refused', 'connection_refused_error']:
        return 'connection_refused'
        
    return failure_str

ipv6_failures = ['address_not_available', 'address_family_not_supported', 'network_unreachable', 'host_unreachable']
def compute_analysis(row):
    failure_str = map_unknown_failure(row['failure_str_raw'])

    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in confirmed_ips:
        return 'dns.confirmed'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in likely_ips:
        return 'dns.likely.blocked'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in false_positive_ips:
        return 'ok'
        
    if row['tls_is_certificate_valid'] == True:
        return 'ok'

    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in confirmed_http_resp:
        return 'http.confirmed'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in likely_http_resp:
        return 'http.likely.blocked'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in false_positive_http_resp:
        return 'ok'
    
    if row['failure_class'] == 'ok':
        return 'ok'

    if row['dns_consistency'] == 'inconsistent':
        return 'dns.inconsistent'
    
    if row['ip_as_org_name'] == 'Bogon':
        return 'dns.bogon'
    
    #if row['dns_blocking_scope'] not in ('u', 'n'):
    #    return f"dns.{row['dns_blocking_scope']}"
    
    if row['tls_is_certificate_valid'] == False:
        return 'tls.bad_cert'
    
    simple_failure = simplify_failure(failure_str)
    if simple_failure in ipv6_failures and row['ip'] and ':' in row['ip']:
        return 'ipv6_error'

    if simple_failure.startswith("ssl_"):
        simple_failure = 'bad_cert'
    
    prefix = row['failure_class']
    if prefix == 'https' and simple_failure.startswith('dns_') or simple_failure == 'nxdomain':
        prefix = 'dns'

    return f'{prefix}.{simple_failure}'

In [47]:
df_ctrl_dns_obs_web = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web
WHERE measurement_start_time > %(measurement_start_day)s
AND probe_cc NOT IN %(cc_list)s
AND tls_is_certificate_valid = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY,
    "cc_list": ANALYSIS_COUNTRY_CODES,
})

In [48]:
%%time
df_ctrl_dns_obs_web_ctrl = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web_ctrl
WHERE measurement_start_time > %(measurement_start_day)s
AND tls_success = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY
})

CPU times: user 809 ms, sys: 93.7 ms, total: 902 ms
Wall time: 32.3 s


In [49]:
from collections import defaultdict
ctrl_map = defaultdict(dict)
for _, row in df_ctrl_dns_obs_web_ctrl.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_ctrl'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_ctrl'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_ctrl'] = set(row['uniq_as_org_name'])
    
for _, row in df_ctrl_dns_obs_web.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_web'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_web'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_web'] = set(row['uniq_as_org_name'])

In [50]:
df_ctrl_dns_obs_web.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[2a06:98c1:3121::3],[13335],[Cloudflare Inc],desk.im
1,"[46.107.16.55, 46.107.16.54, 2001:4c48:16:6::1:20, 2001:4c48:16:6::2:20]",[5483],[Magyar Telekom plc.],videa.hu
2,[185.34.219.59],[198203],[RouteLabel V.O.F.],women.jo
3,"[172.67.179.43, 104.21.31.185]",[13335],[Cloudflare Inc],es.zenit.org
4,"[2606:50c0:8002::153, 185.199.110.153, 185.199.109.153, 2606:50c0:8001::153, 185.199.108.153, 185.199.111.153, 2606:50c0:8000::153, 2606:50c0:8003::153]",[54113],"[Fastly, Inc.]",sibrea.github.io


In [51]:
df_ctrl_dns_obs_web_ctrl.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[91.107.202.180],[24940],[Hetzner Online GmbH],foi.am
1,[154.17.26.106],[906],[DMIT Cloud Services],x18r.com
2,"[104.21.88.99, 2606:4700:3030::ac43:b04d, 2606:4700:3035::6815:5863, 172.67.176.77]",[13335],[Cloudflare Inc],ahang.hu
3,"[2a00:c70:624::935c:f3ce, 147.92.243.206]","[21409, 38631]","[Ikoula Net SAS, LINE Corporation]",line.me
4,"[104.21.38.78, 2606:4700:3035::ac43:dc17, 2606:4700:3036::6815:264e, 172.67.220.23]",[13335],[Cloudflare Inc],taiwantimes.net


In [52]:
def compute_dns_consistency(row):
    if pd.isnull(row['ip']):
        return None
    ctrl = ctrl_map[row['hostname']]
    if len(ctrl):
        return 'unknown'
    if row['ip'] in ctrl.get('uniq_ip_ctrl', {}):        
        return 'consistent.ip_ctrl'
    if row['ip'] in ctrl.get('uniq_ip_web', {}):
        return 'consistent.ip_web'
    if row['ip_asn'] in ctrl.get('uniq_asn_ctrl', {}):
        return 'consistent.asn_ctrl'
    if row['ip_asn'] in ctrl.get('uniq_asn_web', {}):
        return 'consistent.asn_web'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_ctrl', {}):
        return 'consistent.as_org_name_ctrl'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_web', {}):
        return 'consistent.as_org_name_web'
    return 'inconsistent'

In [53]:
df_with_failure['dns_consistency'] = df_with_failure.progress_apply(compute_dns_consistency, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 728363/728363 [00:07<00:00, 94260.54it/s]


In [54]:
df_with_failure['analysis'] = df_with_failure.progress_apply(compute_analysis, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 728363/728363 [00:19<00:00, 38136.15it/s]


In [55]:
df_with_failure.to_csv('2024-ID-df_with_failure.csv')

In [56]:
confirmed_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.confirmed') | 
    (df_with_failure['analysis'] == 'http.confirmed')]

msmt_counts_confirmed = confirmed_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [57]:
likely_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.likely.blocked') |
    (df_with_failure['analysis'] == 'http.likely.blocked')]

msmt_counts_likely = likely_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [58]:
msmt_counts_confirmed.to_csv('2024-indonesia-confirmed-blocked-heuristics.csv')

In [59]:
msmt_counts_likely.to_csv('2024-indonesia-likely-blocked-heuristics.csv')

In [60]:
df_with_failure[
    df_with_failure['failure_str_raw'].str.startswith('unknown_failure')
][['failure_str_raw', 'analysis']].drop_duplicates()

,failure_str_raw,analysis
33361,unknown_failure: local error: tls: bad record MAC,tls.unknown_failure
231910,unknown_failure: remote error: tls: unexpected message,unknown_error
446054,unknown_failure: remote error: tls: bad record MAC,unknown_error
460771,unknown_failure: http2: timeout awaiting response headers,unknown_error
606774,unknown_failure: remote error: tls: unexpected message,tls.unknown_failure


In [61]:
df_with_failure[
    df_with_failure['ip_as_org_name'] == 'Bogon'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_failure,http_response_body_sha1,http_request_url,http_failure,failure_class,failure_str_full,failure_str,failure_str_raw,explorer_url,dns_consistency,analysis
1949,20240401T032459Z_webconnectivity_ID_23700_n1_NXvKNOboPpOXuCtB,http://www.mnr.gov.cn/,20240401032704.230131_ID_webconnectivity_aa417961eb2ed4be,ID,AS23700,Linknet-Fastnet ASN,ID,wifi,2024-04-01 03:27:03,www.mnr.gov.cn,web_connectivity,0.0.0.0,80.0,64666.0,Bogon,45.114.118.156,ID,134451,NewMedia Express Pte Ltd,SG,system,None,0.0.0.0,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.connection_refused,connection_refused,connection_refused,https://explorer.ooni.org/m/20240401032704.230131_ID_webconnectivity_aa417961eb2ed4be,unknown,dns.bogon
1950,20240401T032459Z_webconnectivity_ID_23700_n1_NXvKNOboPpOXuCtB,http://www.mnr.gov.cn/,20240401032704.230131_ID_webconnectivity_aa417961eb2ed4be,ID,AS23700,Linknet-Fastnet ASN,ID,wifi,2024-04-01 03:27:03,www.mnr.gov.cn,web_connectivity,::,80.0,64666.0,Bogon,45.114.118.156,ID,134451,NewMedia Express Pte Ltd,SG,system,None,::,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.connection_refused,connection_refused,connection_refused,https://explorer.ooni.org/m/20240401032704.230131_ID_webconnectivity_aa417961eb2ed4be,unknown,dns.bogon
2387,20240401T032459Z_webconnectivity_ID_23700_n1_NXvKNOboPpOXuCtB,http://www.xinjiang.gov.cn/,20240401032942.642899_ID_webconnectivity_fba97420ab658f50,ID,AS23700,Linknet-Fastnet ASN,ID,wifi,2024-04-01 03:29:42,www.xinjiang.gov.cn,web_connectivity,0.0.0.0,80.0,64666.0,Bogon,45.114.118.156,ID,134451,NewMedia Express Pte Ltd,SG,system,None,0.0.0.0,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.connection_refused,connection_refused,connection_refused,https://explorer.ooni.org/m/20240401032942.642899_ID_webconnectivity_fba97420ab658f50,unknown,dns.bogon
2388,20240401T032459Z_webconnectivity_ID_23700_n1_NXvKNOboPpOXuCtB,http://www.xinjiang.gov.cn/,20240401032942.642899_ID_webconnectivity_fba97420ab658f50,ID,AS23700,Linknet-Fastnet ASN,ID,wifi,2024-04-01 03:29:42,www.xinjiang.gov.cn,web_connectivity,::,80.0,64666.0,Bogon,45.114.118.156,ID,134451,NewMedia Express Pte Ltd,SG,system,None,::,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.connection_refused,connection_refused,connection_refused,https://explorer.ooni.org/m/20240401032942.642899_ID_webconnectivity_fba97420ab658f50,unknown,dns.bogon
3903,20240401T044241Z_webconnectivity_ID_23700_n1_TytvcUl6P8H3Fwj5,http://www.xinjiang.gov.cn/,20240401044339.974385_ID_webconnectivity_8866e87af32350b1,ID,AS23700,Linknet-Fastnet ASN,ID,wifi,2024-04-01 04:43:39,www.xinjiang.gov.cn,web_connectivity,0.0.0.0,80.0,64666.0,Bogon,38.60.171.148,ID,138915,Kaopu Cloud HK Limited,HK,system,None,0.0.0.0,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.connection_refused,connection_refused,connection_refused,https://explorer.ooni.org/m/20240401044339.974385_ID_webconnectivity_8866e87af32350b1,unknown,dns.b

In [104]:
df_with_failure[
    df_with_failure['analysis'] == 'dns.confirmed'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
7024,20240302T031115Z_webconnectivity_VN_7552_n1_Y6YFgIgmts0VsR1V,http://www.tqlcvn.org/,20240302031322.815226_VN_webconnectivity_a92b08d7e018fcf0,VN,7552,Viettel Group,VN,mobile,2024-03-02 03:13:23,www.tqlcvn.org,web_connectivity,125.235.4.59,80.0,7552.0,Viettel Group,27.71.194.194,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
8678,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://scontent-frt3-2.cdninstagram.com/favicon.ico,20240302033631.268289_VN_webconnectivity_23790dba62193cec,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:36:30,scontent-frt3-2.cdninstagram.com,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
9213,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.ms88vtv.com/,20240302033845.448078_VN_webconnectivity_b4dba8dbaf3748ce,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:38:44,www.ms88vtv.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.048580,6.0,2.0,12528.0,288.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,unknown,dns.confirmed
9888,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.vuicungdafa.com/vn/,20240302034111.762361_VN_webconnectivity_3dd4c9c34d1b6218,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:41:10,www.vuicungdafa.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.047289,4.0,2.0,12528.0,292.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,inconsistent,dns.confirmed
58002,20240302T062617Z_webconnectivity_VN_7552_n1_1O3DZ0IQ9xVrd57J,https://vnhrdmay11.org/,20240302062817.607006_VN_webconnectivity_02c6299da4f1f3f0,VN,7552,Viettel Group,VN,wifi,2024-03-02 06:28:15,vnhrdmay11.org,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.71.195.66,VN,75

In [105]:
len(df_with_failure['failure_str_full'].unique())

54

In [106]:
len(df_with_failure['analysis'].unique())

45

In [107]:
df_with_failure[
    (df_with_failure['probe_cc'] == 'VN')
    & (df_with_failure['analysis'] != 'ipv6_error')
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
0,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,204.8.234.161,80.0,395412.0,VS Media Inc.,172.253.4.2,HK,15169,Google LLC,US,system,None,204.8.234.161,1.0,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,ok,ok,,,unknown,ok
1,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,None,NaN,NaN,None,172.253.4.2,HK,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,https://www.gayscape.com/,None,NaN,ok,ok,,,None,ok
2,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,None,NaN,NaN,None,172.253.4.2,HK,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,http://www.gayscape.com/,None,NaN,ok,ok,,,None,ok
3,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,https://www.gaystarnews.com/,20240110000006.989721_VN_webconnectivity_8f2e27c327ff67a5,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:06,www.gaystarnews.com,web_connectivity,104.21.234.125,443.0,13335.0,Cloudflare Inc,172.253.4.2,HK,15169,Google LLC,US,system,None,104.21.234.125,1.0,None,0.085150,2.0,4.0,5552.0,397.0,write_4,1,1.0,CN=gaystarnews.com,gaystarnews.com,"CN=GTS CA 1P5,O=Google Trust Services LLC,C=US",GTS CA 1P5,"[gaystarnews.com, *.gaystarnews.com]",2024-04-07 08:26:03,2024-01-08 08:26:04,3.0,None,None,None,NaN,ok,ok,,,unknown,ok
4,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,https://www.gaystarnews.com/,20240110000006.989721_VN_webconnectivity_8f2e27c327ff67a5,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:06,www.gaystarnews.com,web_connectivity,104.21.234.124,443.0,13335.0,Cloudflare Inc,172.253.4.2,HK,15169,Google LLC,US,system,None,104.21.234.124,1.0,None,0.093481,2.0,4.0,5552.0,397.0,write_4,1,1.0,CN=gaystarnews.com,gaystarnews.com,"CN=GTS CA 1P5,O=Google Trust Services LLC,C=US",GTS CA 1P5,"[gaystarnews.com, *.gaystarnews.com]",2024-04-07 08:26:03,2024-01-08 08:26:04,3.0,None,None,None,NaN,ok,ok,,,unknown,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10069620,20231109T203520Z_webconnectivity_VN_133606_n1_QLEbDWGC6LrxTnox,https://www.vice.com/,20231109210122.902658_VN_webconnectivity_397a21ea447095d8,VN,133606,Viettel Timor Leste,TL,wifi,2023-11-09 21:01:12,www.vice.com,web_connectivity,151.101.66.133,443.0,54113.0,"Fastly, Inc.",172.253.215.197,BE,15169,Google LLC,US,syst

In [70]:
df_with_failure[
     (df_with_failure['analysis'] != 'ok')
]['analysis'].unique()

array(['unknown_error', 'dns.inconsistent', 'dns.confirmed',
       'tcp.connection_refused', 'tcp.timeout', 'tls.bad_cert',
       'tls.connection_reset', 'ipv6_error', 'dns.likely.blocked',
       'dns.bogon', 'tls.timeout', 'tcp.connection_aborted',
       'tls.eof_error', 'tls.unknown_failure', 'tcp.permission_denied',
       'http.confirmed', 'tls.connection_aborted',
       'tcp.network_unreachable', 'tls.tls_handshake_failure',
       'tcp.host_unreachable', 'tls.internal_error'], dtype=object)

In [87]:
%%time
target_domains = [
    "www.reddit.com"
]
df_special = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_certificate_chain_length,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'ID' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 11.9 s, sys: 829 ms, total: 12.7 s
Wall time: 1min 24s


In [88]:
len(df_special)

15589

In [89]:
df_special['dns_consistency'] = df_special.progress_apply(compute_dns_consistency, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15589/15589 [00:00<00:00, 103798.41it/s]


In [90]:
df_special['analysis'] = df_special.progress_apply(compute_analysis, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15589/15589 [00:00<00:00, 39856.95it/s]


In [92]:
df_special["probe_asn"] = "AS" + df_special["probe_asn"].apply(str)

In [93]:
method_of_blocking = df_special[df_special['analysis'] != 'ok'].pivot_table(index=['analysis'], values=['measurement_uid'], columns=['probe_asn','probe_as_org_name'],
                                                aggfunc={'measurement_uid': len}).fillna(0).reset_index()

method_of_blocking

analysis              measurement_uid  \
probe_asn                                                         AS131111   
probe_as_org_name                             PT Mora Telematika Indonesia   
0                                   dns.bogon                          0.0   
1                               dns.confirmed                         79.0   
2                  dns.dns_server_misbehaving                          0.0   
3                          dns.likely.blocked                         79.0   
4                                dns.nxdomain                          0.0   
5                                 dns.timeout                          0.0   
6                              https.bad_cert                        108.0   
7                    https.connection_refused                          0.0   
8                      https.connection_reset                          0.0   
9                             https.eof_error                          0.0   
10                     https.host_unreachable                          0.0   
11                       https.internal_error                          0.0   
12                       https.invalid_record                          0.0   
13                  https.network_unreachable                          0.0   
14                              https.timeout                          8.0   
15                                 ipv6_error                          0.0   
16                     tcp.connection_refused                          0.0   
17                      tcp.permission_denied                          0.0   
18                                tcp.timeout                          0.0   
19                               tls.bad_cert                          0.0   
20                       tls.connection_reset                          0.0   
21                                tls.timeout                          0.0   

                                                                  \
probe_asn                         AS131706              AS131775   
probe_as_org_name PT SELARAS CITRA TERABIT PT. Jupiter Jala Arta   
0                                      0.0                   0.0   
1                                      0.0                   0.0   
2                                      0.0                   0.0   
3                                     13.0                   0.0   
4                                      0.0                   0.0   
5                                      0.0                   0.0   
6                                      0.0                   0.0   
7                                     13.0                   0.0   
8                                      0.0                   0.0   
9                                      0.0                   0.0   
10                                     0.0                   0.0   
11                                     0.0                   0.0   
12                                     0.0                   0.0   
13                                     0.0                   0.0   
14                                     0.0                   0.0   
15                                     0.0                   5.0   
16                                     0.0                   0.0   
17                                     0.0                   0.0   
18                                     0.0                   0.0   
19                                     0.0                   0.0   
20                                     0.0                   0.0   
21                                     0.0                   0.0   

                                                             \
probe_asn                   AS132641               AS132652   
probe_as_org_name PT Netciti Persada PT. Cahaya Buana Raksa   
0                                0.0                    0.0   
1                                0.0                    0.0   
2                                0.0                    0.0   
3      